# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232770642519                   -0.50    7.0
  2   -7.249210533082       -1.78       -1.38    1.0
  3   -7.251081475301       -2.73       -1.84    4.0
  4   -7.250995691288   +   -4.07       -1.89    3.0
  5   -7.251299631065       -3.52       -2.36    2.0
  6   -7.251336308257       -4.44       -2.90    2.0
  7   -7.251338742499       -5.61       -3.82    3.0
  8   -7.251338782216       -7.40       -3.92    4.0
  9   -7.251338797095       -7.83       -4.34    3.0
 10   -7.251338797793       -9.16       -4.58    1.0
 11   -7.251338798695       -9.04       -5.59    4.0
 12   -7.251338798701      -11.22       -5.68    2.0
 13   -7.251338798704      -11.59       -6.10    2.0
 14   -7.251338798705      -12.19       -6.76    3.0
 15   -7.251338798705      -13.71       -7.08    2.0
 16   -7.251338798705      -14.05       -7.58    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04465387678484341
[ Info: Arnoldi iteration step 2: normres = 0.7571087833922114
[ Info: Arnoldi iteration step 3: normres = 0.6448882381239458
[ Info: Arnoldi iteration step 4: normres = 0.46281788549199593
[ Info: Arnoldi iteration step 5: normres = 0.2559710815746281
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.23e-02, 6.93e-02, 1.12e-01, 2.15e-01, 2.66e-02)
[ Info: Arnoldi iteration step 6: normres = 0.47179993796639264
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.23e-02, 1.07e-01, 4.46e-01, 5.47e-02, 8.26e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0760227570879972
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.98e-04, 1.47e-02, 2.70e-02, 1.45e-02, 5.94e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08653100389648162
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.84e-